In [19]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [1]:
# Open DataFrames
df = pd.read_csv('df_10.csv')

# Show first 5 rows
df.head()

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


,Unnamed: 0,index,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,...,Total_Votes,Helpful_Percentage,Review_Length,Sentence_Length,Word_Length,Helpful_Rating,Helpful_Votes_Scaled,Helpful_Percentage_Book_Score,Badge,Helpful?
0,14,34,000100039X,"[81, 92]",5.0,This is one of the first (literary) books I re...,"06 24, 2000",A1NPNGWBVD9AK3,"Bruce Kendall ""BEK""",Simple Wisdom,...,92,0.880435,1542,15.000000,4.258865,0.921901,0.993132,0.587119,4,1
1,18,43,000100039X,"[8, 10]",5.0,The Prophet is Kahlil Gibran's best known work...,"10 27, 2009",A3IS4WGMFR4X65,"Colin Lewis ""WisdomNote""",The most loving book ever written,...,10,0.800000,2294,22.705882,4.883289,0.816553,0.851511,0.545942,4,1
2,19,46,000100039X,"[8, 10]",5.0,Gibran Khalil Gibran was born in 1883 in what ...,"01 10, 2006",AWLFVCT9128JV,"Dave_42 ""Dave_42""",The Lessons Of Life,...,10,0.800000,712,19.428571,4.103704,0.816553,0.851511,0.545942,4,1
3,35,75,000100039X,"[10, 12]",5.0,"Certainly the words are of Kahlil Gibran, but ...","11 5, 2001",A1SP45I55GQIIE,harendra desai,divine wisdom in earthly words.,...,12,0.833333,700,21.000000,4.675000,0.851313,0.889774,0.547093,4,1
4,36,76,000100039X,"[0, 13]",2.0,"I evidently misread the writeup, I thought it ...","05 3, 2009",A2ZB1G1KUE6OS6,Harley,Unexpected purchase,...,13,0.000000,178,11.666667,3.885714,0.010826,0.000000,0.541325,1,0


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 787981 entries, 0 to 787980
Data columns (total 22 columns):
Unnamed: 0                       787981 non-null int64
index                            787981 non-null int64
asin                             787981 non-null object
helpful                          787981 non-null object
overall                          787981 non-null float64
reviewText                       787981 non-null object
reviewTime                       787981 non-null object
reviewerID                       787981 non-null object
reviewerName                     786144 non-null object
summary                          787978 non-null object
unixReviewTime                   787981 non-null int64
Helpful_Votes                    787981 non-null int64
Total_Votes                      787981 non-null int64
Helpful_Percentage               787981 non-null float64
Review_Length                    787981 non-null int64
Sentence_Length                  787981 non-null floa

In [11]:
df['Helpful'] = df['Helpful?']

In [4]:
corpus = df['reviewText']

import nltk
import re
nltk.download('stopwords')

wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)
norm_corpus = normalize_corpus(corpus)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/coreyjwade/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
def make_xy(df, vectorizer):
    vectorizer = vectorizer
    X = vectorizer.fit_transform(df.reviewText)
    X = X.tocsc()  # some versions of sklearn return COO format
    y = df.Helpful
    return X, y

def make_xy_norm(df, vectorizer):
    vectorizer = vectorizer
    X = vectorizer.fit_transform(norm_corpus)
    X = X.tocsc()  # some versions of sklearn return COO format
    y = df.Helpful
    return X, y

In [7]:
vectorizers = [CountVectorizer(), CountVectorizer(ngram_range=(1,2)), CountVectorizer(ngram_range=(1,3)), TfidfVectorizer(), TfidfVectorizer(ngram_range=(1, 2)), TfidfVectorizer(ngram_range=(1, 3))]

In [8]:
xy_options = [make_xy, make_xy_norm]

In [26]:
# keras_regression_test requires "from sklearn.model_selection import train_test_split"

def keras_classification_test(X, y, numbers=[128,64], activation='relu', optimizer='adam', loss='categorical_crossentropy'):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    # Save the number of columns in predictors: n_cols
    n_cols = X_train.shape[1]
    input_shape = (n_cols,)

    # Set up the model: model
    model = Sequential()
    
    # Add the first layer
    model.add(Dense(numbers[0], activation=activation, input_shape=(n_cols,)))
    
    # Add addition layers
    for i in range(len(numbers)-1):
        model.add(Dense(numbers[i+1], activation=activation))

    # Add the output layer
    model.add(Dense(2, activation='softmax'))

    # Compile the model
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

    # Define early_stopping_monitor
    early_stopping_monitor = EarlyStopping(patience=2)

    # Fit the model
    model.fit(X_train, y_train, validation_split=0.3, epochs=30, callbacks=[early_stopping_monitor])
    
    # Get score for predictions
    score = model.evaluate(X_test, y_test)
    
    # Print score
    print(score)

In [14]:
df_original = df.copy

In [15]:
df = df.sample(frac=0.1)

In [22]:
len(df)

78798

In [17]:
X, y = make_xy(df, CountVectorizer())

In [25]:
keras_classification_test(X, y, numbers=[128,64])

ValueError: Error when checking target: expected dense_9 to have shape (2,) but got array with shape (1,)

In [28]:
y

384353    1
71953     1
571828    1
594157    1
272707    1
549100    1
212902    1
552421    1
603870    1
48005     0
182919    1
449762    0
200246    1
696535    0
607837    1
152807    1
292325    1
201063    0
101283    1
27097     1
629296    0
785585    1
244890    0
246891    1
190576    1
128668    1
198394    1
586239    1
431330    1
780146    1
         ..
472921    1
742451    1
95201     0
109876    1
499732    0
280931    1
1189      0
11728     1
759751    1
137038    1
410851    1
246193    1
338664    1
473799    1
146883    1
462816    1
609249    1
148666    1
483011    1
241660    1
47499     0
128013    0
780431    1
179432    1
663803    1
689297    1
35888     0
569493    1
683807    0
67002     0
Name: Helpful, Length: 78798, dtype: int64